# LoanPrediction -  Krediyi ödeme şansı daha düşük olan müşterileri belirleme

### İmport the Data

In [ ]:
import pandas as pd #ython programlama dilinde Pandas kütüphanesini içe aktarma
import numpy as np # Python programlama dilinde NumPy kütüphanesini içe aktarma

import warnings
warnings.filterwarnings('ignore') # tüm uyarı mesajlarının görmezden gelme

import matplotlib.pyplot as plt # Python'da veri görselleştirme

from sklearn.model_selection import train_test_split #  bir veri setini eğitim ve test setlerine ayırma
from sklearn.ensemble import RandomForestClassifier #, birden fazla karar ağacından oluşur ve bu ağaçların sonuçlarını birleştirerek daha iyi tahminler elde etme
from sklearn.metrics import classification_report, accuracy_score # Model tahmini
from sklearn.preprocessing import LabelEncoder #Kategorik değişkenleri sayısal değerlere dönüştürme
# Python'da makine öğrenimi uygulamaları için kullanılan scikit-learn kütüphanesinden çeşitli bileşenleri içe aktarma

In [ ]:
df=pd.read_csv('LoansTrainingSet.csv')

### EDA -Exploratory Data Analysis

In [ ]:
df

In [ ]:
df.shape;

In [ ]:
df.info()

In [ ]:
df.describe() # Temel istatistiksel özet

In [ ]:
df.corr(numeric_only=True) # Sayısal sütunalrı arasındaki korelasyon matrisi

In [ ]:
df.isnull().sum() # Eksikm değerlerin sayıısnı hesaplar

In [ ]:
df.columns

### Data Data Preparation

In [ ]:
df = df.drop_duplicates() #Tekrarları kaldırma

In [ ]:
df

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median()) # sayısal sütunlar için eksik değerleri orta değer ile atama 

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns # kategorik sütunları bulma

In [ ]:
df[categorical_cols] = df[categorical_cols].apply(
    lambda col: col.fillna(col.mode().iloc[0]) 
) # kategorik sütunlara tekrarlanan veriyi atama

In [ ]:
print(df.isnull().sum());

In [ ]:
df

In [ ]:
df['Loan Status'].unique()

In [ ]:
df['Term'].unique()

In [ ]:
df['Home Ownership'].unique()

In [ ]:
df[categorical_cols] = df[categorical_cols].apply(lambda col: LabelEncoder().fit_transform(col.astype(str)))

 d={'Charged Off':0,'Fully Paid':1},
 d={'Short Term':0,'Long Term':1}----yapmamıza gerek yok apply ile yaptık

In [ ]:
df[categorical_cols]

Özellik Mühendisliği: Kredi Kullanım Oranı

In [ ]:
df['Credit_Utilization'] = df['Current Credit Balance'] / df['Current Loan Amount'].replace(0, np.nan)

Eğitim ve Test Setlerine Ayır

In [ ]:
x = df.drop(['Loan Status', 'Loan ID', 'Customer ID'], axis=1)  # Özellikler
y = df[['Loan Status']]  # Hedef değişken

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Modelleme

In [ ]:
model = RandomForestClassifier(random_state=42)
r=RandomForestClassifier()

In [ ]:
r.fit(x_train, y_train)  # Modeli eğit

In [ ]:
rtahmin = r.predict(x_test)
print("Accuracy:", accuracy_score(y_test, rtahmin))
print(classification_report(y_test, rtahmin))

Soru Oluşturulması

In [ ]:
importances = r.feature_importances_
feature_names = x.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

 Önemli sorular oluştur

In [ ]:
questions = importance_df['Feature'].head(5).apply(lambda x: f"What is your {x.replace('_', ' ').lower()}?")

In [ ]:
print("\nQuestions to ask the customer:")
print('\n'.join(questions))